<a href="https://colab.research.google.com/github/MD03/RLT/blob/main/RLT_DP_Policy_Improvement_and_Value_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DP- Policy Improvement and Value Iteration
import numpy as np

class MDP:
    def __init__(self, num_states, num_actions, transition_probs, rewards, gamma=0.9):
        self.num_states = num_states
        self.num_actions = num_actions
        self.transition_probs = transition_probs  # shape: (num_states, num_actions, num_states)
        self.rewards = rewards  # shape: (num_states, num_actions)
        self.gamma = gamma

    def policy_evaluation(self, policy, tol=1e-6):
        V = np.zeros(self.num_states)
        while True:
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                action = policy[s]
                V[s] = sum(self.transition_probs[s, action, s_prime] *
                           (self.rewards[s, action] + self.gamma * V[s_prime])
                           for s_prime in range(self.num_states))
                delta = max(delta, abs(v - V[s]))
            if delta < tol:
                break
        return V

    def policy_improvement(self, V):
        policy = np.zeros(self.num_states, dtype=int)
        for s in range(self.num_states):
            policy[s] = np.argmax(self._bellman_operator(s, V))
        return policy

    def value_iteration(self, tol=1e-6):
        V = np.zeros(self.num_states)  # Initialize value function
        while True:
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                V[s] = max(self._bellman_operator(s, V))
                delta = max(delta, abs(v - V[s]))
            if delta < tol:
                break
        policy = self.policy_improvement(V)
        return V, policy

    def _bellman_operator(self, state, V):
        Q = np.zeros(self.num_actions)
        for a in range(self.num_actions):
            for s_prime in range(self.num_states):
                Q[a] += self.transition_probs[state, a, s_prime] * (self.rewards[state, a] + self.gamma * V[s_prime])
        return Q

# Example usage
num_states = 3
num_actions = 2
transition_probs = np.array([[[0.5, 0.5, 0.0], [1.0, 0.0, 0.0]],
                             [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]],
                             [[0.0, 1.0, 0.0], [0.5, 0.5, 0.0]]])
rewards = np.array([[1.0, 2.0], [0.0, 0.0], [5.0, -1.0]])

mdp = MDP(num_states, num_actions, transition_probs, rewards)
V, policy = mdp.value_iteration()
print("Optimal value function:", V)
print("Optimal policy:", policy)